Author: Tarun Ruchandani (tarun.ruchandani@columbia.edu)
Date: 2016-08-02 14:26:09 -0700 (Tue, 08 Aug 2016)
Revision: 1

Spark LDA for NASA ADS v1.0 

To Do:
x Generate document term matrix for LDA
x Get LDA Results on small Corpus

8/3:

x Generate a JS Matrix per mapequation's input
- Generate topic trend reports.
o Build Mapequation with JS Matrix
o Extend to a larger corpus
o Build Docker Container
o Deploy to adsqb

- JS Matrix: figure out the right implementation - also try ifa library


8/5:

o Take care of no abstract situation
o Scale to 500 papers

8/8:

o Take care of no abstract situation
o Scale to 500 papers
o Create a graph

8/24:
o Scale to 6000 papers


In [2]:
import numpy as np
import textmining
import pyspark
import lda
import lda.datasets
import ads
import pandas as pd
from textblob import TextBlob
from numpy  import array
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import json

In [6]:
# # Extract ApJ and OpJ papers

# ads.config.token = 'TkEI7jQxScyoCtzHfhcpCWVelRqySmH5XBksQCFA'

# papers_ApJ = list(ads.SearchQuery(bibstem='ApJ',start=100,rows=10000))
# # papers_OPJ = list(ads.SearchQuery(bibstem='OPJ',start=100,rows=1000))
# # papers_PhyEs = list(ads.SearchQuery(bibstem='PhyEs',start=100,rows=1000))

# # papers = papers_ApJ+papers_OPJ+papers_PhyEs
# papers = papers_ApJ

# paper_abstracts=list()

# for paper in papers:
#     paper_abstracts.append(paper.abstract)

# # Removing papers with null abstract

# paper_abstracts = filter(None, paper_abstracts)
# paper_abstracts = list(paper_abstracts)
# print(len(paper_abstracts))


/Users/tarunruchandani/anaconda3/lib/python3.5/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'abstract', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


477


In [6]:
ads.config.token = 'TkEI7jQxScyoCtzHfhcpCWVelRqySmH5XBksQCFA'

fo = open('articles.json', 'w')
papers_ApJ = list(ads.SearchQuery(bibstem='ApJ', start=100,rows=10000, fl=['abs','bibcode', 'title', 'body']))

papers = papers_ApJ

for paper in papers:
    fo.write(json.dumps(paper._raw))
    fo.write("\n")
fo.close()

# and to read it
from ads.search import Article

fi = open('articles.json', 'r')
docs = []
for line in fi:
    docs.append(Article(**json.loads(line)))
    

print(len(docs))

2000


/Users/tarunruchandani/anaconda3/lib/python3.5/site-packages/ads/search.py:437: UserWarning: Response rows did not match input rows. Setting this query's rows to 2000
  "Setting this query's rows to {}".format(self.query['rows']))


In [22]:
paper_abstracts = filter(None, [x.abs for x in docs])
# paper_abstracts = list(docs)
print(len(docs))


2000


In [27]:
from ads.search import Article
a = Article(**paper._raw)
# a = Article(author = paper._raw['author'],...)
a._raw

{'abs': None,
 'abstract': None,
 'bibcode': '1898ApJ.....8..173M',
 'id': '70041',
 'title': ['The K lines of beta Aurigae.']}

In [29]:
# Build Term-Document Matrix

tdm = textmining.TermDocumentMatrix()

for paper in paper_abstracts:
    tdm.add_doc(paper)
    
paper_abstracts_tdm = list()

for row in tdm.rows(cutoff=0):
    paper_abstracts_tdm.append(row)

paper_abstracts_tdm_df = pd.DataFrame(paper_abstracts_tdm)
paper_abstracts_tdm_df.head()

# Vocab of terms in abstracts
vocab = paper_abstracts_tdm_df._slice(slice(1))
vocab

d_t_freq = paper_abstracts_tdm_df._slice(slice(1,1411))
# d_t_freq = paper_abstracts_tdm_df
b=d_t_freq.values
b=b.astype(int)


In [26]:
# running LDA on TDM
# Parameters:	
# rdd – RDD of documents, which are tuples of document IDs and term (word) count vectors. 
# The term count vectors are “bags of words” with a fixed-size vocabulary 
# (where the vocabulary size is the length of the vector). Document IDs must be unique and >= 0.

# k – Number of topics to infer, i.e., the number of soft cluster centers. (default: 10)

# maxIterations – Maximum number of iterations allowed. (default: 20)

# docConcentration – Concentration parameter (commonly named “alpha”) 
# for the prior placed on documents’ distributions over topics (“theta”). (default: -1.0)

# topicConcentration – Concentration parameter (commonly named “beta” or “eta”) 
# for the prior placed on topics’ distributions over terms. (default: -1.0)

# seed – Random seed for cluster initialization. Set as None to generate seed based on system time. (default: None)

# checkpointInterval – Period (in iterations) between checkpoints. (default: 10)

# optimizer – LDAOptimizer used to perform the actual calculation. Currently “em”, “online” are supported. 
# (default: “em”)

model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
model.fit(b)

topic_word = model.topic_word_  # model.components_ also works


for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(vocab)


Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'
ValueError: x must be strictly positive
Exception ignored in: 'lda._lda.lgamma'


In [25]:
doc_topic = model.doc_topic_
print("type(doc_topic): {}".format(type(doc_topic)))
print("shape: {}".format(doc_topic.shape))

type(doc_topic): <class 'numpy.ndarray'>
shape: (0, 20)


In [14]:
# Document-topic Probab

doc_topic = model.doc_topic_

for n in range(5):
    sum_pr = sum(doc_topic[n,:])
    print("document: {} sum: {}".format(n, sum_pr))

for n in range(10):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n...".format(n,
                                            topic_most_pr
                                            ))


document: 0 sum: 0.9999999999999999
document: 1 sum: 0.9999999999999997
document: 2 sum: 1.0000000000000002
document: 3 sum: 1.0000000000000002
document: 4 sum: 1.0000000000000002
doc: 0 topic: 10
...
doc: 1 topic: 3
...
doc: 2 topic: 13
...
doc: 3 topic: 7
...
doc: 4 topic: 12
...
doc: 5 topic: 17
...
doc: 6 topic: 16
...
doc: 7 topic: 17
...
doc: 8 topic: 8
...
doc: 9 topic: 14
...


In [15]:
# Building JS Matrix - JSD Implementation 1

def multi_js(p, q):
    """Jensen-Shannon divergence (symmetric) between two multinomials,
    expressed in nats."""
    if (len(q.shape) == 2):
        axis = 1
    else:
        axis = 0
    # D_{JS}(P\|Q) = (D_{KL}(P\|Q) + D_{KL}(Q\|P)) / 2
    return 0.5 * ((q * (np.log(q.clip(1e-10,1))
                        - np.log(p.clip(1e-10,1)))).sum(axis)
                      + (p * (np.log(p.clip(1e-10,1))
                              - np.log(q.clip(1e-10,1)))).sum(axis))


JS_D1D2 = multi_js(doc_topic[0],doc_topic[1])
JS_D1D2

3.7927952980509869

In [16]:
# Alternative JS implementation

def jsd(x,y): #Jensen-shannon divergence
    import warnings
    warnings.filterwarnings("ignore", category = RuntimeWarning)
    x = np.array(x)
    y = np.array(y)
    d1 = x*np.log2(2*x/(x+y))
    d2 = y*np.log2(2*y/(x+y))
    d1[np.isnan(d1)] = 0
    d2[np.isnan(d2)] = 0
    d = 0.5*np.sum(d1+d2)    
    return d

jsd(doc_topic[0],doc_topic[1])

0.69611336429598514

In [22]:
# Constructing JS Matrix

# Creates a list containing 5 lists, each of 8 items, all set to 0
w, h = 500, 500 
Matrix = [[0 for x in range(w)] for y in range(h)] 

JSM = list()
js_df = pd.DataFrame()

for index1,item1 in enumerate(doc_topic):
    for index2,item2 in enumerate(doc_topic):
        result = jsd(doc_topic[index1],doc_topic[index2])
        Matrix[index1][index2] = result
        temp_list = list()        
        temp_list = (index1,index2,result)
        JSM.append(temp_list)

# print(JSM)

text_file3 = open("/Users/tarunruchandani/Desktop/HarvardSummer2016/Infomap/JS_output3.txt", "w")
text_file3.write(str(JSM))
text_file3.close()

In [21]:
# Writing to a txt file for Mapequation

text_file = open("/Users/tarunruchandani/Desktop/HarvardSummer2016/Infomap/JS_output2.txt", "w")

for index1,item1 in enumerate(doc_topic):
    for index2,item2 in enumerate(doc_topic):
        result = 1-(jsd(doc_topic[index1],doc_topic[index2]))
        text_file.write("%s %s %s\n" %(index1, index2, result))

text_file.close()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline 

G=nx.Graph()
